In [1]:
import pandas as pd
import json

In [2]:
epl_data = ("./Resources/english-premier-league-match-data/datafile 2/season17-18/season_stats.json")
fifa_player_info = ("./Resources/fifa-18-demo-player-dataset/PlayerPersonalData.csv")
fifa_player_info_df = pd.read_csv(fifa_player_info)

In [3]:
with open(epl_data) as jsonfile:
    match_stats_json = json.load(jsonfile)

In [136]:
# getting the name, team, position, and match rating for each player for each match. real match data

player_details = []

for match, team in match_stats_json.items():
    for team2,agg_stats in team.items():
        for x,y in agg_stats['team_details'].items():
            if x == 'team_name':
                team_name = y
        for player_name, player_stats in agg_stats['Player_stats'].items():
            save_stats=[]
            save_stats.append(player_name)
            save_stats.append(team_name)
            for key,value in player_stats['player_details'].items():
                if key == 'player_position_info':
                    save_stats.append(value)
                if key == 'player_rating':
                    save_stats.append(value)
            player_details.append(save_stats)

In [137]:
# put the dataframe together from the EPL .json

columns = ['name','team','position','match rating']
epl_data_df = pd.DataFrame(player_details, columns = columns)
epl_data_df.dtypes

name            object
team            object
position        object
match rating    object
dtype: object

In [148]:
# epl_data_df['club'] = epl_data_df['club'].astype(str)
# epl_data_df['team'] = epl_data_df['team'].astype(str)
epl_data_df['name'] = epl_data_df['name'].astype(str)
epl_data_df['match rating'] = epl_data_df['match rating'].astype(float)

In [162]:
# checking the match rating values of a player

epl_data_df.sort_values('name').head()

,name,team,position,match rating
3024,Aaron Cresswell,West Ham,DC,5.91
12708,Aaron Cresswell,West Ham,DML,7.28
6229,Aaron Cresswell,West Ham,DC,7.02
4861,Aaron Cresswell,West Ham,DC,6.38
7972,Aaron Cresswell,West Ham,DC,5.96


In [149]:
# creating a dataframe that we'll merge with the grouped dataframe

epl_data_name_team = epl_data_df[['name','team']]

In [200]:
# getting the average match rating for each player for the season

match_ratings = epl_data_df.groupby('name', as_index = False).mean()
match_ratings.head()

,name,match rating
0,Aaron Cresswell,6.508649
1,Aaron Lennon,6.467586
2,Aaron Mooy,6.868889
3,Aaron Ramsdale,0.000000
4,Aaron Ramsey,7.271250


In [206]:
# merging the DFs together to get the average rating, club for each player

match_ratings_club = match_ratings.merge(epl_data_name_team, on='name')

In [207]:
match_ratings_club.tail()

,name,match rating,team
13670,Álvaro Morata,6.935484,Chelsea
13671,Álvaro Morata,6.935484,Chelsea
13672,Álvaro Morata,6.935484,Chelsea
13673,Álvaro Morata,6.935484,Chelsea
13674,Álvaro Morata,6.935484,Chelsea


In [208]:
match_ratings_club = match_ratings_club.drop_duplicates('name')

In [209]:
len(match_ratings_club)

607

In [45]:
# dataframe with the FIFA video game ratings for all players

fifa_player_info_df = fifa_player_info_df[['ID','Name','Overall','Potential','Club']]
fifa_player_info_df.head()
len(fifa_player_info_df)

17981

In [43]:
# get the EPL teams from '17 -'18

wiki_tables = pd.read_html('https://en.wikipedia.org/wiki/2017%E2%80%9318_Premier_League')
epl_teams = wiki_tables[1]
epl_teams = epl_teams.loc[0:19,'Team']
epl_teams

0                    Arsenal
1                Bournemouth
2     Brighton & Hove Albion
3                    Burnley
4                    Chelsea
5             Crystal Palace
6                    Everton
7          Huddersfield Town
8             Leicester City
9                  Liverpool
10           Manchester City
11         Manchester United
12          Newcastle United
13               Southampton
14                Stoke City
15              Swansea City
16         Tottenham Hotspur
17                   Watford
18      West Bromwich Albion
19           West Ham United
Name: Team, dtype: object

In [53]:
# FIFA Data: get only the players that are in the EPL

epl_players = fifa_player_info_df.loc[fifa_player_info_df['Club'].isin(epl_teams)]

In [69]:
# FIFA Data

epl_players.head()

,ID,Name,Overall,Potential,Club
6,193080,De Gea,90,92,Manchester United
7,183277,E. Hazard,90,91,Chelsea
11,192985,K. De Bruyne,89,92,Manchester City
12,192119,T. Courtois,89,92,Chelsea
13,184941,A. Sánchez,89,89,Arsenal


In [72]:
len(epl_players)

650

In [75]:
# splitting the FIFA game data by last name

new = epl_players["Name"].str.rsplit(" ", n = 1, expand = True)  
epl_players['last_name'] = new[1]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [169]:
# create a unique key for the FIFA game data
epl_players['key'] = epl_players['last_name'] + epl_players['Club']

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [197]:
epl_players.sort_values('key').head()

,ID,Name,Overall,Potential,Club,last_name,key
1732,186190,P. van Aanholt,75,76,Crystal Palace,Aanholt,AanholtCrystal Palace
3433,222513,R. Aarons,72,79,Newcastle United,Aarons,AaronsNewcastle United
3114,231352,T. Abraham,73,86,Swansea City,Abraham,AbrahamSwansea City
2200,155976,C. Adam,74,74,Stoke City,Adam,AdamStoke City
12482,222104,T. Adarabioyo,63,83,Manchester City,Adarabioyo,AdarabioyoManchester City


In [210]:
# splitting the EPL .json data by last name

new2 = match_ratings_club['name'].str.rsplit(" ", n = 1, expand = True)  
match_ratings_club['last_name'] = new2[1]

In [211]:
# create a unique key for the EPL .json data
match_ratings_club['key'] = match_ratings_club['last_name'] + match_ratings_club['team']

In [212]:
match_ratings_club.sort_values('key').head()

,name,match rating,team,last_name,key
10411,Patrick van Aanholt,5.286111,Crystal Palace,Aanholt,AanholtCrystal Palace
11213,Rolando Aarons,2.277273,Newcastle United,Aarons,AaronsNewcastle United
12415,Tammy Abraham,6.023636,Swansea,Abraham,AbrahamSwansea
2161,Charlie Adam,2.174688,Stoke,Adam,AdamStoke
12855,Tosin Adarabioyo,0.000000,Manchester City,Adarabioyo,AdarabioyoManchester City


In [213]:
merged_df = pd.merge(epl_players, match_ratings_club, on='key',how='inner')

In [214]:
len(merged_df)

646

In [215]:
merged_df

,ID,Name,Overall,Potential,Club,last_name_x,key,name,match rating,team,last_name_y
0,193080,De Gea,90,92,Manchester United,Gea,GeaManchester United,David De Gea,6.697333,Manchester United,Gea
1,193080,De Gea,90,92,Manchester United,Gea,GeaManchester United,David de Gea,6.925000,Manchester United,Gea
2,183277,E. Hazard,90,91,Chelsea,Hazard,HazardChelsea,Eden Hazard,7.469714,Chelsea,Hazard
3,223938,K. Hazard,63,73,Chelsea,Hazard,HazardChelsea,Eden Hazard,7.469714,Chelsea,Hazard
4,192985,K. De Bruyne,89,92,Manchester City,Bruyne,BruyneManchester City,Kevin De Bruyne,7.597895,Manchester City,Bruyne
5,192119,T. Courtois,89,92,Chelsea,Courtois,CourtoisChelsea,Thibaut Courtois,6.630571,Chelsea,Courtois
6,153079,S. Agüero,89,89,Manchester City,Agüero,AgüeroManchester City,Sergio Agüero,6.732759,Manchester City,Agüero
7,176635,M. Özil,88,88,Arsenal,Özil,ÖzilArsenal,Mesut Özil,6.977778,Arsenal,Özil
8,215914,N. Kanté,87,90,Chelsea,Kanté,KantéChelsea,N'Golo Kanté,7.242059,Chelsea,Kanté
9,195864,P. Pogba,87,92,Manchester United,Pogba,PogbaManchester United,Paul Pogba,7.478889,Manchester United,Pogba
